In [29]:
import pandas as pd
import numpy as np

# Import the first CSV file
feature_df = pd.read_csv('ufc_features.csv')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
feature_df.head()

,p1_fighter,p2_fighter,referee,winner,p1_height,p1_weight,p1_reach,p1_stance,p1_SLpM,p1_Str. Acc.,p1_SApM,p1_Str. Def,p1_TD Avg.,p1_TD Acc.,p1_TD Def.,p1_Sub. Avg.,p2_height,p2_weight,p2_reach,p2_stance,p2_SLpM,p2_Str. Acc.,p2_SApM,p2_Str. Def,p2_TD Avg.,p2_TD Acc.,p2_TD Def.,p2_Sub. Avg.,p1_age_at_event,p2_age_at_event,height_diff,reach_diff,weight_diff,age_diff,slpm_diff,stracc_diff,sapm_diff,strdef_diff,tdavg_diff,tdacc_diff,tddef_diff,subavg_diff,p1_days_since_last_fight,p2_days_since_last_fight,days_since_last_fight_diff,p1_wins,p1_losses,p1_total,p2_wins,p2_losses,p2_total,win_diff,loss_diff,total_diff,p1_win_streak,p2_win_streak,p1_age_adjusted_SLpM,p2_age_adjusted_SLpM,p1_age_adjusted_Str. Acc.,p2_age_adjusted_Str. Acc.,p1_age_adjusted_SApM,p2_age_adjusted_SApM,p1_age_adjusted_Str. Def,p2_age_adjusted_Str. Def,p1_age_adjusted_TD Avg.,p2_age_adjusted_TD Avg.,p1_age_adjusted_TD Acc.,p2_age_adjusted_TD Acc.,p1_age_adjusted_TD Def.,p2_age_adjusted_TD Def.,p1_age_adjusted_Sub. Avg.,p2_age_adjusted_Sub. Avg.,p1_KD_EMA,p2_KD_EMA,p1_SIG_STR_PCT_EMA,p2_SIG_STR_PCT_EMA,p1_TD_PCT_EMA,p2_TD_PCT_EMA,p1_SUB_ATT_EMA,p2_SUB_ATT_EMA,p1_REV_EMA,p2_REV_EMA,p1_CTRL_EMA,p2_CTRL_EMA,p1_R1_KD_EMA,p2_R1_KD_EMA,p1_R1_SIG_STR_PCT_EMA,p2_R1_SIG_STR_PCT_EMA,p1_R1_TD_PCT_EMA,p2_R1_TD_PCT_EMA,p1_R1_SUB_ATT_EMA,p2_R1_SUB_ATT_EMA,p1_R1_REV_EMA,p2_R1_REV_EMA,p1_R1_CTRL_EMA,p2_R1_CTRL_EMA,p1_SIG_STR_PCT_DETAILED_EMA,p2_SIG_STR_PCT_DETAILED_EMA,p1_R1_SIG_STR_PCT_DETAILED_EMA,p2_R1_SIG_STR_PCT_DETAILED_EMA,p1_SIG_STR_LANDED_EMA,p2_SIG_STR_LANDED_EMA,p1_SIG_STR_ATTEMPTED_EMA,p2_SIG_STR_ATTEMPTED_EMA,p1_TOTAL_STR_LANDED_EMA,p2_TOTAL_STR_LANDED_EMA,p1_TOTAL_STR_ATTEMPTED_EMA,p2_TOTAL_STR_ATTEMPTED_EMA,p1_TD_LANDED_EMA,p2_TD_LANDED_EMA,p1_TD_ATTEMPTED_EMA,p2_TD_ATTEMPTED_EMA,p1_R1_SIG_STR_LANDED_EMA,p2_R1_SIG_STR_LANDED_EMA,p1_R1_SIG_STR_ATTEMPTED_EMA,p2_R1_SIG_STR_ATTEMPTED_EMA,p1_R1_TOTAL_STR_LANDED_EMA,p2_R1_TOTAL_STR_LANDED_EMA,p1_R1_TOTAL_STR_ATTEMPTED_EMA,p2_R1_TOTAL_STR_ATTEMPTED_EMA,p1_R1_TD_LANDED_EMA,p2_R1_TD_LANDED_EMA,p1_R1_TD_ATTEMPTED_EMA,p2_R1_TD_ATTEMPTED_EMA,p1_HEAD_LANDED_EMA,p2_HEAD_LANDED_EMA,p1_HEAD_ATTEMPTED_EMA,p2_HEAD_ATTEMPTED_EMA,p1_BODY_LANDED_EMA,p2_BODY_LANDED_EMA,p1_BODY_ATTEMPTED_EMA,p2_BODY_ATTEMPTED_EMA,p1_LEG_LANDED_EMA,p2_LEG_LANDED_EMA,p1_LEG_ATTEMPTED_EMA,p2_LEG_ATTEMPTED_EMA,p1_DISTANCE_LANDED_EMA,p2_DISTANCE_LANDED_EMA,p1_DISTANCE_ATTEMPTED_EMA,p2_DISTANCE_ATTEMPTED_EMA,p1_CLINCH_LANDED_EMA,p2_CLINCH_LANDED_EMA,p1_CLINCH_ATTEMPTED_EMA,p2_CLINCH_ATTEMPTED_EMA,p1_GROUND_LANDED_EMA,p2_GROUND_LANDED_EMA,p1_GROUND_ATTEMPTED_EMA,p2_GROUND_ATTEMPTED_EMA,p1_R1_HEAD_LANDED_EMA,p2_R1_HEAD_LANDED_EMA,p1_R1_HEAD_ATTEMPTED_EMA,p2_R1_HEAD_ATTEMPTED_EMA,p1_R1_BODY_LANDED_EMA,p2_R1_BODY_LANDED_EMA,p1_R1_BODY_ATTEMPTED_EMA,p2_R1_BODY_ATTEMPTED_EMA,p1_R1_LEG_LANDED_EMA,p2_R1_LEG_LANDED_EMA,p1_R1_LEG_ATTEMPTED_EMA,p2_R1_LEG_ATTEMPTED_EMA,p1_R1_DISTANCE_LANDED_EMA,p2_R1_DISTANCE_LANDED_EMA,p1_R1_DISTANCE_ATTEMPTED_EMA,p2_R1_DISTANCE_ATTEMPTED_EMA,p1_R1_CLINCH_LANDED_EMA,p2_R1_CLINCH_LANDED_EMA,p1_R1_CLINCH_ATTEMPTED_EMA,p2_R1_CLINCH_ATTEMPTED_EMA,p1_R1_GROUND_LANDED_EMA,p2_R1_GROUND_LANDED_EMA,p1_R1_GROUND_ATTEMPTED_EMA,p2_R1_GROUND_ATTEMPTED_EMA
0,Cory Sandhagen,Deiveson Figueiredo,Dan Miragliotta,1,71.0,135.0,70.0,Switch,5.02,0.44,3.46,0.56,1.19,0.32,0.63,0.3,65.0,135.0,68.0,Orthodox,2.91,0.54,3.55,0.49,1.62,0.34,0.60,1.4,33.034908,37.374401,6.0,2.0,0.0,-4.339493,2.11,-0.10,-0.09,0.07,-0.43,-0.02,0.03,-1.1,273.0,161.0,112.0,10,4,14,13,5,18,-3,-1,-4,0,0,0.151960,0.077861,0.013319,0.014448,0.104738,0.094985,0.016952,0.013111,0.036023,0.043345,0.009687,0.009097,0.019071,0.016054,0.009081,0.037459,0.012025,0.299520,0.427808,0.499030,0.295141,0.250166,0.266130,0.319192,0.032290,0.000000,369.136117,174.851985,0.008057,0.001953,0.434544,0.846666,0.664548,0.467068,0.016114,0.037895,0.000000,0.000000,62.763841,98.073334,0.427808,0.499030,0.434544,0.846666,73.462980,44.916534,168.352377,94.296907,112.881890,50.330579,222.104682,99.836223,2.220167,

In [30]:
#prepare data for training

# Drop the columns
columns_to_drop = ['p1_fighter', 'p2_fighter'] #method
feature_df = feature_df.drop(columns=columns_to_drop)
cols_to_drop = [col for col in feature_df.columns if col.startswith('method_')]
feature_df.drop(columns=cols_to_drop, inplace=True)


# Clean all column names
def clean_column_name(col):
    return col.lower().replace(' ', '_').replace('.', '').replace('-', '_')

# Apply to all columns
feature_df.columns = [clean_column_name(col) for col in feature_df.columns]

# Identify all categorical columns
categorical_cols = ['p1_stance', 'p2_stance']

# One-hot encode all categorical variables
feature_df = pd.get_dummies(feature_df, columns=categorical_cols)

feature_df = feature_df.sample(frac=1, random_state=42).reset_index(drop=True)

# encode the referee using frequency
ref_counts = feature_df['referee'].value_counts()
feature_df['referee_freq'] = feature_df['referee'].map(ref_counts)
feature_df.drop(columns=['referee'], inplace=True)


In [31]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# target
X = feature_df.drop(columns=['winner'])
y = feature_df['winner']

# train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# identify numeric columns for scaling
numeric_cols = X_train.select_dtypes(include=['float64', 'int64']).columns.tolist()

# impute numeric NaNs with median
num_imputer = SimpleImputer(strategy='median')
X_train[numeric_cols] = num_imputer.fit_transform(X_train[numeric_cols])
X_test[numeric_cols]  = num_imputer.transform(X_test[numeric_cols])

# scale numeric features
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled  = X_test.copy()

X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test_scaled[numeric_cols]  = scaler.transform(X_test[numeric_cols])

results = {}

# 1) Logistic Regression (on scaled)
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_train_scaled, y_train)
logreg_pred = logreg.predict(X_test_scaled)
results['Logistic Regression'] = {
    'accuracy': accuracy_score(y_test, logreg_pred),
    'report': classification_report(y_test, logreg_pred)
}

# 2) Random Forest (on raw)
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
results['Random Forest'] = {
    'accuracy': accuracy_score(y_test, rf_pred),
    'report': classification_report(y_test, rf_pred)
}

# 3) XGBoost (on raw)
import xgboost as xgb
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)
results['XGBoost'] = {
    'accuracy': accuracy_score(y_test, xgb_pred),
    'report': classification_report(y_test, xgb_pred)
}

# 4) LightGBM (on raw)
import lightgbm as lgb
lgb_model = lgb.LGBMClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42
)
lgb_model.fit(X_train, y_train)
lgb_pred = lgb_model.predict(X_test)
results['LightGBM'] = {
    'accuracy': accuracy_score(y_test, lgb_pred),
    'report': classification_report(y_test, lgb_pred)
}

# 5) CatBoost (on raw)
from catboost import CatBoostClassifier
cat_model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    depth=5,
    random_seed=42,
    verbose=False
)
cat_model.fit(X_train, y_train)
cat_pred = cat_model.predict(X_test)
results['CatBoost'] = {
    'accuracy': accuracy_score(y_test, cat_pred),
    'report': classification_report(y_test, cat_pred)
}

# Print evaluation
for name, res in results.items():
    print(f"\n{name}:")
    print(f"  Accuracy: {res['accuracy']:.4f}")
    print(f"  Classification Report:\n{res['report']}")

# --- feature importances across tree models ---
feature_importance = pd.DataFrame({
    'RF':    rf.feature_importances_,
    'XGB':   xgb_model.feature_importances_,
    'LGBM':  lgb_model.feature_importances_,
    # you can include CatBoost too if you like:
    'CAT':   cat_model.get_feature_importance()
}, index=X.columns)

# sort by, say, XGBoost importances (or pick any column)
feature_importance = feature_importance.sort_values(by='XGB', ascending=False)

print("\nFeature importances (all features, sorted by XGBoost):")
print(feature_importance)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4125, number of negative: 2415
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007008 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 35568
[LightGBM] [Info] Number of data points in the train set: 6540, number of used features: 173
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.630734 -> initscore=0.535367
[LightGBM] [Info] Start training from score 0.535367
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

In [32]:
# Export the model
xgb_model.save_model('xgb_model.json') 